# Creating Containers

## Best Practices before we start:
- Keep track of all commands you use during your installation either by writing them down or creating a copy of your “history”.
- To copy history issue the following: “history >> history.log”
- Always use “sudo” before issuing a singularity command
- If you are building your container on the platform, always use “screen”
- You may need to install it in your instance “yum install screen”
- Use the screen command before starting any work. This will keep your environment setup and commands running in case you lose your connection to the instance.
- You can reattach your “screen” by issuing the “screen -r” command after getting disconnected.
- You can manually detach your screen by using “ctrl+a+d” again “screen -r” will reattach it to your current session. 
- It’s better to build using a “centos” os vs ubuntu for your container.


### Creating a container in sandbox mode

Create a writable folder and add os image

In [ ]:
!sudo singularity build --sandbox centos/ library://centos

Connect to the container by using the singularity shell command
- the -w flag is important as it enables write mode for the sandbox. IE it saves all your changes

In [ ]:
!sudo singularity shell -w centos/

You'll need to setup your environment here. The following are typical a good starting point.\
MPICH can be swapped OpenMPI in case you want a different mpi flavor. 


In [ ]:
!yum -y update
!yum -y groupinstall "Development Tools" "Development Libraries"
!yum -y install gcc gcc-gfortran gcc-c++ git wget m4 mpich python3

In this step we're downloading the software we want to build, SU2, via get into our sandbox\
In addtion we're moving into the directory to start our build.\
It's important to note, that if you place an install file in your home directory on the host machine\
singularity will automatically bind the home directory into the container as the home directory as well.\
This means you can leave a file in your home directory and access it directly inside the container as well. 

In [ ]:
!git clone https://github.com/su2code/SU2.git 
%cd SU2/

Here we are just starting the build process to install SU2.\ 
Steps will vary depending on the software you are compiling

In [ ]:
!./meson.py build -Denable-autodiff=true -Dwith-mpi=enabled  

In [ ]:
!./ninja -j 18 -C build install

### These get added to .bashrc or .bash_profile. Note that these paths can change if you change the installation path as well. 
---
`export SU2_RUN=/usr/local/bin
export SU2_HOME=/root/SU2
export PATH=$PATH:$SU2_RUN
export PYTHONPATH=$PYTHONPATH:$SU2_RUN
`

In [2]:
!echo export SU2_RUN=/usr/local/bin >> ~/.bash_profile
!echo export SU2_HOME=/root/SU2 >> ~/.bash_profile
!echo export PATH=$PATH:$SU2_RUN >> ~/.bash_profile
!echo export PYTHONPATH=$PYTHONPATH:$SU2_RUN >> ~/.bash_profile
!cat ~/test.test

SyntaxError: invalid syntax (<ipython-input-2-929fd1f89c11>, line 2)

Exit the sandbox!

In [ ]:
!exit

Now we're going to turn our sandbox into a container.\
command syntax is\
`singularity build nameofcontainer.sif /path/to/<sandbox directory>
`
\
We're also going to move our container to your work directory so it'll save\
when you shut this job down.

In [ ]:
!sudo singularity build su2.sif centos/
!cp su2.sif ~/work/shared/
%cd ..

## Create a container from a recipe

You can also create a container from a recipe file.\
Typically we want to keep track of the installation commands you used in the sandbox mode\
so that you can use those to create a recipe file.\
\
There are several pieces to a definition file.\
Please see the following link for more details:\
[Singularity Definition File](https://sylabs.io/guides/3.5/user-guide/definition_files.html)


In [ ]:
!singularity build su2-recipe.sif su2.def